Personalized pricing. -> heterogeneous price elasticity of demand for people with different income level, learning which users would respond most strongly to a small discount.

Boost overall revenue.

1. `DML` based estimators -> estimate heterogeneous price sensitivities
2. `SingleTreeCateInterpreter` -> presentation-ready summar of the key features that explain the biggest differences in responsiveness to a discount
3. `SingleTreePolicyInterpreter` -> recommends policy on who should receive a discount in order to increase revenue

In [1]:
import os
import urllib.request
import numpy as np
import pandas as pd

from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor

from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter

import matplotlib.pyplot as plt

/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behavio

In [2]:
file_url = "https://msalicedatapublic.z5.web.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_url)

In [3]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand
0,3,53,1.834234,2,8,1,1,4.903237,0.960863,1.0,3.917117
1,5,54,7.171411,7,9,0,1,3.330161,0.732487,1.0,11.585706
2,3,33,5.351920,6,9,0,1,3.036203,1.130937,1.0,24.675960
3,2,34,6.723551,0,8,0,1,7.911926,0.929197,1.0,6.361776
4,4,30,2.448247,5,8,1,0,7.148967,0.533527,0.8,12.624123


In [4]:
train_data.describe()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,3.00450,38.912800,5.008411,3.503900,10.025000,0.498700,0.798700,5.051504,1.128157,0.91257,15.493496
std,1.41226,12.465061,1.993379,2.303631,2.227391,0.500023,0.400992,2.862219,0.597024,0.08531,6.568161
min,1.00000,18.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.184644,0.80000,3.000000
25%,2.00000,28.000000,3.637598,1.000000,9.000000,0.000000,1.000000,2.973419,0.709357,0.80000,9.128451
50%,3.00000,39.000000,5.002157,3.500000,10.000000,0.000000,1.000000,5.007432,0.989363,0.90000,15.299043
75%,4.00000,50.000000,6.352960,6.000000,12.000000,1.000000,1.000000,7.032429,1.403052,1.00000,20.471066
max,5.00000,60.000000,13.129260,7.000000,18.000000,1.000000,1.000000,15.931413,6.156005,1.00000,27.923607


In [5]:
Y = train_data["demand"] #outcome
T = train_data["price"] # intervention
X = train_data[["income"]] #features
W = train_data.drop(columns=["demand", "price", "income"]) #confounders


In [6]:
# Test data
X_test = np.linspace(0, 5, 100).reshape(-1, 1)
X_test_data = pd.DataFrame(X_test, columns=["income"])

# Parametric heterogeneity

In [7]:
log_T = np.log(T)
log_Y = np.log(Y)

# Train econML model
est = LinearDML(
    model_y=GradientBoostingRegressor(),
    model_t=GradientBoostingRegressor(),
    featurizer=PolynomialFeatures(degree=2, include_bias=False)
)

est.fit(log_Y, log_T, X=X, W=W, inference="statsmodels")
# Statsmodels ou boostrap
#https://econml.azurewebsites.net/_autosummary/econml.dml.LinearDML.html

te_pred = est.effect(X_test)
te_pred_interval = est.effect_interval(X_test)

In [8]:
plt.figure(figsize=(10, 6))
plt.plot(X_test.flatten(), te_pred, label="Sales Elasticity Prediction")
plt.fill_between(
    X_test.flatten(),
    te_pred_interval[0],
    te_pred_interval[1],
    alpha=.2,
    label="95% CI"
)
plt.xlabel("Income")
plt.ylabel("Songs Sales Elasticity")
plt.title("Songs Sales Elasticity")

plt.show()

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [9]:
est.summary()

,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
income,2.417,0.072,33.723,0.0,2.276,2.557
income^2,-0.432,0.024,-17.733,0.0,-0.479,-0.384
,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,-3.015,0.045,-67.259,0.0,-3.103,-2.927


# Nonparametric Heterogeneity

In [10]:
est = CausalForestDML(
    model_y = GradientBoostingRegressor(),
    model_t = GradientBoostingRegressor()
)

est.fit(log_Y, log_T, X=X, W=W, inference="blb")
te_pred = est.effect(X_test)
te_pred_interval = est.effect_interval(X_test)

# Understand Treatment Effects with EconML

- `SingleTreeCateInterpreter`: trains a single DT on the treatment effects outputted by any EconML estimator.

In [15]:
intrp = SingleTreeCateInterpreter(include_model_uncertainty=True, max_depth=2, min_samples_leaf=10)
intrp.interpret(est, X_test)
plt.figure(figsize=(25, 5))
intrp.plot(feature_names=X.columns, fontsize=12)
plt.savefig("interpret.png")

# Make Policy Decision

In [17]:
intrp = SingleTreePolicyInterpreter(risk_level=.05, max_depth=2, min_samples_leaf=1, min_impurity_decrease=.001)
intrp.interpret(est, X_test, sample_treatment_costs=-1)
plt.figure(figsize=(25, 5))
intrp.plot(feature_names=X.columns, treatment_names=["Discount", "Non-Discount"], fontsize=12)